## 구글 드라이브와 연결

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# koreanize-matplotlib 라이브러리 설치
!pip install koreanize-matplotlib

# matplotlib 라이브러리, matplotlib에서 한글을 사용하기 위한 라이브러리 임포트
import matplotlib.pyplot as plt
import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.5 MB/s eta 0:00:00


In [13]:
# 마운트된 드라이브 경로 확인
import os
project_path = '/content/gdrive/My Drive/sw융합프로젝트(1)/'

# 해당 폴더 내 파일 목록 확인
files = os.listdir(project_path)
print(files)

['행정구역(읍면동)', '행정구역(시군구)', '인천광역시_시내버스 정류소 현황_20241231.csv', '인천광역시.geojson', '아파트(매매)_실거래가_20250512141056.csv', '연립다세대(전월세)_실거래가_20250512142423.csv', '아파트(전월세)_실거래가_20250512142547.csv', '연립다세대(매매)_실거래가_20250512142554.csv', 'apt_geocoded.csv', 'multi_geocoded.csv']


In [14]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

In [5]:
import pandas as pd

# 파일 경로 설정 (Google Drive 또는 로컬 경로에 맞게 수정)
csv_path = '/content/gdrive/My Drive/sw융합프로젝트(1)/인천광역시_시내버스 정류소 현황_20241231.csv'

# 1. CSV 불러오기
df = pd.read_csv(csv_path, encoding='utf-8')  # 한글깨짐 방지
# 위도랑 경도 제대로 고치기
df = df.rename(columns={'위도': '경도_tmp', '경도': '위도'})
df = df.rename(columns={'경도_tmp': '경도'})
df.head()

,기준 일자,정류소 명,정류소 번호,정류소아이디,권역,행정동 명,엑스좌표,경도,와이좌표,위도
0,2024-12-31,(구)주안2동치안센터,37302.0,163000302,미추홀구,주안2동,170842.1600,126.671236,439486.1420,37.457078
1,2024-12-31,(주)경동세라믹스,89146.0,168001146,서구,오류왕길동,166087.8384,126.616978,450837.1724,37.559186
2,2024-12-31,(주)경인양행앞,42096.0,168000096,서구,가좌1동,170343.6370,126.665414,444145.8737,37.499045
3,2024-12-31,(주)경인양행앞,42097.0,168000097,서구,석남2동,170465.3876,126.666789,444178.4131,37.499342
4,2024-12-31,(주)대한특수금속,39050.0,165000050,남동구,논현고잔동,172879.0446,126.694491,433147.5666,37.400029


In [6]:
# 2. 서구만 필터링
df['권역'] = df['권역'].str.strip()
df = df[df['권역'] == '서구']

df['geometry'] = df.apply(lambda row: Point(row['경도'], row['위도']), axis=1)

gdf_seogu_bus = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [7]:
# 평균 중심 계산
center_lat = gdf_seogu_bus['위도'].mean()  # 실제 위도
center_lon = gdf_seogu_bus['경도'].mean()  # 실제 경도

m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

for _, row in gdf_seogu_bus.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],  # 위도, 경도 순서!
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=row['정류소 명']
    ).add_to(m)

m

In [8]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point

# 3. 위도/경도 형변환 및 결측 제거
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# 4. GeoDataFrame 변환 (Point(경도, 위도))
df['geometry'] = df.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
gdf_bus = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# 5. 행정동별 정류장 수 집계
dong_counts = gdf_bus['행정동 명'].value_counts().reset_index()
dong_counts.columns = ['행정동 명', '정류장 수']
dong_counts

,행정동 명,정류장 수
0,청라3동,97
1,오류왕길동,95
2,검암경서동,91
3,아라동,68
4,연희동,65
5,청라2동,62
6,신현원창동,61
7,청라1동,49
8,가정1동,44
9,가좌1동,44


In [9]:
gdf_seogu_bus

,기준 일자,정류소 명,정류소 번호,정류소아이디,권역,행정동 명,엑스좌표,경도,와이좌표,위도,geometry
1,2024-12-31,(주)경동세라믹스,89146.0,168001146,서구,오류왕길동,166087.8384,126.616978,450837.1724,37.559186,POINT (126.61698 37.55919)
2,2024-12-31,(주)경인양행앞,42096.0,168000096,서구,가좌1동,170343.6370,126.665414,444145.8737,37.499045,POINT (126.66541 37.49905)
3,2024-12-31,(주)경인양행앞,42097.0,168000097,서구,석남2동,170465.3876,126.666789,444178.4131,37.499342,POINT (126.66679 37.49934)
8,2024-12-31,(주)스킨이데아,89388.0,168001388,서구,원찬동,166105.6856,126.617473,444479.2189,37.501903,POINT (126.61747 37.5019)
9,2024-12-31,(주)스킨이데아,89389.0,168001389,서구,원창동,166107.5171,126.617495,444459.0933,37.501721,POINT (126.6175 37.50172)
...,...,...,...,...,...,...,...,...,...,...,...
6856,2024-12-31,효성아파트,42168.0,168000168,서구,석남1동,170282.1244,126.664651,445792.3371,37.513878,POINT (126.66465 37.51388)
6876,2024-12-31,휴먼시아후문,42885.0,168000885,서구,청라2동,168749.6769,126.647243,447554.8589,37.529707,POINT (126.64724 37.52971)
6891,2024-12-31,희성금속,42018.0,168000018,서구,가좌3동,171276.1861,126.676031,442303.9651,37.482480,POINT (126.67603 37.48248)
6892,2024-12-31,힐데스하임,42876.0,168000876,서구,청라1동,169240.1662,126.652792,447562.8537,37.529796,POINT (126.65279 37.5298)


In [10]:
import geopandas as gpd
import folium
from shapely.geometry import Point

# 1. 행정동 경계 geojson 불러오기
dong_geo_path = '/content/gdrive/My Drive/sw융합프로젝트(1)/인천광역시.geojson'
gdf_dong = gpd.read_file(dong_geo_path)
gdf_dong['sggnm'] = gdf_dong['sggnm'].str.strip()
gdf_dong = gdf_dong[gdf_dong['sggnm'] == '서구']

# 2. 행정동 명 추출 및 병합용 컬럼 생성
gdf_dong['dong_name'] = gdf_dong['adm_nm'].str.extract(r'(\S+동)')

# 3. dong_counts 컬럼명 정리 및 병합
dong_counts.columns = ['dong_name', '정류장 수']
gdf_merge = gdf_dong.merge(dong_counts, on='dong_name', how='left')
gdf_merge['정류장 수'] = gdf_merge['정류장 수'].fillna(0)
gdf_merge['dong_name'] = gdf_merge['dong_name'].astype(str)

# 4. GeoDataFrame 재정의
gdf_merge = gpd.GeoDataFrame(gdf_merge, geometry='geometry', crs=gdf_dong.crs)

# 5. 지도 생성
m = folium.Map(location=[37.545, 126.675], zoom_start=13)

# 6. Choropleth 층 추가
folium.Choropleth(
    geo_data=gdf_merge.__geo_interface__,
    data=gdf_merge,
    columns=['dong_name', '정류장 수'],
    key_on='feature.properties.dong_name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='버스 정류장 수'
).add_to(m)

# 7. 툴팁 및 경계선 추가
folium.GeoJson(
    gdf_merge,
    name="행정동 경계",
    tooltip=folium.GeoJsonTooltip(
        fields=["adm_nm", "정류장 수"],
        aliases=["행정동", "정류장 수"],
        localize=True
    ),
    style_function=lambda x: {
        "fillOpacity": 0,
        "color": "gray",
        "weight": 1
    }
).add_to(m)

# 8. 지도 출력
m


In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

# 2. 위도/경도 숫자화 및 결측 제거
gdf_seogu_bus['위도'] = pd.to_numeric(gdf_seogu_bus['위도'], errors='coerce')
gdf_seogu_bus['경도'] = pd.to_numeric(gdf_seogu_bus['경도'], errors='coerce')
gdf_seogu_bus = gdf_seogu_bus.dropna(subset=['위도', '경도'])

# 3. 정류장 데이터를 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(gdf_seogu_bus['경도'], gdf_seogu_bus['위도'])]
gdf_bus = gpd.GeoDataFrame(gdf_seogu_bus, geometry=geometry, crs='EPSG:4326')

# 4. 행정동 GeoJSON 불러오기
dong_geo_path = '/content/gdrive/My Drive/sw융합프로젝트(1)/인천광역시.geojson'
gdf_dong = gpd.read_file(dong_geo_path)
gdf_dong['sggnm'] = gdf_dong['sggnm'].str.strip()
gdf_dong = gdf_dong[gdf_dong['sggnm'] == '서구']

# 5. 공간 조인: 정류장이 포함된 행정동 찾기
gdf_joined = gpd.sjoin(gdf_bus, gdf_dong, how='inner', predicate='within')

# 6. 행정동별 정류장 수 집계
dong_counts = gdf_joined['adm_nm'].value_counts().reset_index()
dong_counts.columns = ['adm_nm', '정류장 수']

# 7. 병합
gdf_merge = gdf_dong.merge(dong_counts, on='adm_nm', how='left')
gdf_merge['정류장 수'] = gdf_merge['정류장 수'].fillna(0)

# 8. Choropleth 시각화
m = folium.Map(location=[37.545, 126.675], zoom_start=13)

folium.Choropleth(
    geo_data=gdf_merge.__geo_interface__,
    data=gdf_merge,
    columns=['adm_nm', '정류장 수'],
    key_on='feature.properties.adm_nm',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='버스 정류장 수'
).add_to(m)

# 9. 툴팁 추가
folium.GeoJson(
    gdf_merge,
    name="행정동 경계",
    tooltip=folium.GeoJsonTooltip(
        fields=["adm_nm", "정류장 수"],
        aliases=["행정동", "정류장 수"],
        localize=True
    ),
    style_function=lambda x: {
        "fillOpacity": 0,
        "color": "gray",
        "weight": 1
    }
).add_to(m)

m.save('서구_정류장_자동추정_지도.html')
m

In [12]:
print(gdf_joined.head())
print(gdf_joined.crs, gdf_dong.crs)
print(gdf_merge[['adm_nm', '정류장 수']].sort_values('정류장 수', ascending=False).head())


        기준 일자      정류소 명   정류소 번호     정류소아이디  권역  행정동 명         엑스좌표  \
1  2024-12-31  (주)경동세라믹스  89146.0  168001146  서구  오류왕길동  166087.8384   
2  2024-12-31   (주)경인양행앞  42096.0  168000096  서구   가좌1동  170343.6370   
3  2024-12-31   (주)경인양행앞  42097.0  168000097  서구   석남2동  170465.3876   
8  2024-12-31   (주)스킨이데아  89388.0  168001388  서구    원찬동  166105.6856   
9  2024-12-31   (주)스킨이데아  89389.0  168001389  서구    원창동  166107.5171   

           경도         와이좌표         위도                    geometry  \
1  126.616978  450837.1724  37.559186  POINT (126.61698 37.55919)   
2  126.665414  444145.8737  37.499045  POINT (126.66541 37.49905)   
3  126.666789  444178.4131  37.499342  POINT (126.66679 37.49934)   
8  126.617473  444479.2189  37.501903   POINT (126.61747 37.5019)   
9  126.617495  444459.0933  37.501721   POINT (126.6175 37.50172)   

   index_right  OBJECTID          adm_nm   adm_cd     adm_cd2    sgg sido  \
1          109       873  인천광역시 서구 검암경서동  2308051  2826051500  28260   28  